In [237]:
import pandas as pd
import numpy as np
from datetime import datetime
import os


from pathlib import Path

Path('covid.db').touch()

In [238]:
import sqlite3
conn = sqlite3.connect('covid.db')
c = conn.cursor()




In [239]:
# create a dictionary with all three CSV's
covid_data = { "states" : { "file" : "Resources/us-states.csv",
                    "schema" : '''CREATE TABLE states (date text, state text, cases int, deaths int)'''},
         "prison" : { "file" : "Resources/covid_prison_cases.csv",
                    "schema" : '''CREATE TABLE prison (name text, total_staff_deaths int, total_prisoner_cases int, total_prisoner_deaths int, as_of_date text )'''},
         "deaths" : {"file" :"Resources/3_covid_tracking_project_historical_testing_numbers_and_covid_deaths_by_state.csv",
                               "schema" : '''CREATE TABLE deaths (stage text, date text, total_population int, cumulative_deaths int,deaths_increase int, last_update_et text)'''}
       }




In [240]:
#create a DataFrame for all three csv's

for k in covid_data:
    d = pd.read_csv(covid_data[k]["file"])
    covid_data[k]["df"] = d
    
states_df = covid_data["states"]["df"]
prison_df = covid_data["prison"]["df"]
deaths_df = covid_data["deaths"]["df"]



In [241]:
# Keep the columns that needed for the analysis
states_df = states_df.loc[:, ("date","state","cases","deaths")]
prison_df = prison_df.loc[:, ("name","total_prisoner_cases","total_prisoner_deaths","as_of_date")]
deaths_df = deaths_df.loc[:, ("state","date","cumulative_deaths")]

In [242]:
#rename columns 
states_df.rename(columns = {"date" : "Date","state":"State","cases":"Number of Cases", "deaths":"Number of Deaths"}, inplace = True)
prison_df.rename(columns = {"as_of_date":"Date", "name" : "State", "total_prisoner_cases":"Total Prisoner Cases","total_prisoner_deaths":"Total Prisoner Deaths", }, inplace = True)
deaths_df.rename(columns = {"date":"Date","state" : "State","cumulative_deaths":"Cumulative Prisoner Deaths"}, inplace = True)


In [243]:
# Reorganizing the columns
states_df = states_df[["Date","State", "Number of Cases", "Number of Deaths"]]
prison_df = prison_df[["Date", "State", "Total Prisoner Cases","Total Prisoner Deaths"]]
deaths_df = deaths_df[["Date","State","Cumulative Prisoner Deaths"]]

In [244]:
# drop the rows with missing data
clean_states_df = states_df.dropna(how = 'any')
clean_prison_df = prison_df.dropna(axis = 0, how = 'any')
clean_deaths_df = deaths_df.dropna(axis = 0, how = 'any')

In [245]:
# change the date format

clean_prison_df[('Date')] = pd.to_datetime(clean_prison_df[('Date')]).dt.strftime('%Y-%m-%d')
clean_states_df[('Date')] = pd.to_datetime(clean_states_df[('Date')]).dt.strftime('%Y-%m-%d')
clean_deaths_df[('Date')] = pd.to_datetime(clean_deaths_df[('Date')]).dt.strftime('%Y-%m-%d')


/Users/sarapatel/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sarapatel/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [246]:
# create an index
clean_deaths_df = clean_deaths_df.set_index(['State', 'Date'])
clean_states_df = clean_states_df.set_index(['State', 'Date'])
clean_prison_df = clean_prison_df.set_index(['State', 'Date'])


In [247]:
# Check to see if there are any rows with missing data
# clean_death_df.count()
# clean_states_df.count()
# clean_prison_df.count()

In [254]:
# sort columns
clean_deaths_df = clean_deaths_df.sort_values(['State', 'Date'], ascending=False)
clean_states_df = clean_states_df.sort_values(['State', 'Date'], ascending=False)
clean_prison_df = clean_prison_df.sort_values(['State', 'Date'], ascending=False)



In [255]:
# hej not sure if we should merge them here or in the postgress?
# same for teh calculations?


In [257]:
clean_deaths_df.to_csv("Resources/cleaned_deaths.csv")

In [258]:
clean_states_df.to_csv("Resources/cleaned_deaths.csv")

In [259]:
clean_prison_df.to_csv("Resources/cleaned_deaths.csv")